Also look at binomial significance? maybe start as motivation, then show general concept? also compare with rule of thumb formulas?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats

In [ ]:
def nll_hist(obs, exp):
    """
    Negative log-likelihood for histogram with poisson distributed counts (up to constant terms)
    """
    return np.sum(exp, axis=0) - np.sum(obs * np.log(exp), axis=0)

In [ ]:
def tau_from_db(b, db):
    """
    Calculate tau (the ratio between expected background in the off and on region)
    from the expected background and the absolute uncertainty on it.
    """
    return b / (db ** 2)

In [ ]:
b = 5
delta_b = 2

nobs = 11

p-value for rejecting the null hypothesis of no signal?

In [ ]:
non = nobs
tau = tau_from_db(b, delta_b)
noff = tau * b

In [ ]:
np.array([np.array([1, 2]), np.array([3, 4])]).sum(axis=0)

In [ ]:
def nll_onoff(non, noff, s, b, tau):
    obs = np.array([non, noff])
    exp = np.array([s + b, tau * b])
    return nll_hist(obs, exp)

In [ ]:
nll_onoff(non, noff, s=0, b=b, tau=tau)

unconditional fit, allowing signal (alternative hypothesis):

In [ ]:
minimize(lambda pars: nll_onoff(non, noff, s=pars[0], b=pars[1], tau=tau), (1, 1))

makes sense - 6, 5

conditional fit, assuming 0 signal (null hypothesis):

In [ ]:
minimize(lambda pars: nll_onoff(non, noff, s=0, b=pars[0], tau=tau), (1,))

background gets slightly "pulled", but disagreement with observations

This setup has the nice property that we don't need to do a fit since we can find the maximum likelihood estimates (MLEs) analytically:

In [ ]:
import sympy
from sympy.solvers import solve

In [ ]:
non_, noff_, s_, b_, tau_ = sympy.symbols("n_on n_off s b tau")

In [ ]:
nll = (s_ + b_) + (tau_ * b_) - (non_ * sympy.log(s_ + b_) + noff_ * sympy.log(tau_ * b_))
nll

In [ ]:
s_hat = solve(sympy.diff(nll, s_), s_)[0]
s_hat

In [ ]:
b_hat = solve(sympy.diff(nll.subs(s_, s_hat), b_), b_)[0]
b_hat

These two are intuitively very clear - without constraint the best-fit signal yield will just be the total number of "on" events minus the number of expected background and the background will be exclusively determined from the "off" region.

The best-fit background for a fixed signal is less clear and we will get 2 solutions for the quadratic equation that results from setting the derivative to 0:

In [ ]:
b_hathat = solve(sympy.diff(nll, b_), b_)
display(b_hathat[0])
display(b_hathat[1])

Here we only need the case for `s=0`:

In [ ]:
display(b_hathat[0].subs(s_, 0))
display(b_hathat[1].subs(s_, 0))

We can simplify the expression under the square root and see that the first solution is 0 which is not a useful estimate, so we only need the second solution.

The relevant MLEs to get the log-likelihood ratio test statistic are therefore:

In [ ]:
def mles(non, noff, b, tau):
    "Maximum likelihood estimates for the on-off likelihood"
    shat = non - b
    bhat = noff / tau
    bhathat = (noff + non) / (tau + 1)
    return shat, bhat, bhathat

In [ ]:
mles(non, noff, b, tau) # consistent with fit above

In [ ]:
def nllr(non, noff, tau):
    """
    Negative log likelihood ratio for on-off problem
    """
    b = noff / tau
    shat, bhat, bhathat = mles(non, noff, b, tau)

    cond_nll = nll_onoff(non, noff, s=0, b=bhathat, tau=tau)
    uncond_nll = nll_onoff(non, noff, s=shat, b=bhat, tau=tau)

    return np.where(
        (shat <= 0) | (bhat <= 0),
        0, # we choose to view lower counts as background expectation not as evidence against signal
        cond_nll - uncond_nll
    )

In [ ]:
nllr_obs = nllr(non, noff, tau)
nllr_obs

Is this significant? Let's throw toys under null hypothesis

In [ ]:
def toy_nllr():
    non = np.random.poisson(b)
    noff = np.random.poisson(tau * b)
    return nllr(non, noff, tau)

In [ ]:
np.random.poisson(b, size=n_toys)

In [ ]:
n_toys = 100000
toys = nllr(np.random.poisson(b, size=n_toys), np.random.poisson(tau * b, size=n_toys), tau)

In [ ]:
toys

In [ ]:
plt.hist(toys, bins=100);
plt.axvline(nllr_obs, color="red")
plt.yscale("log")

In [ ]:
pvalue = (toys >= nllr_obs).mean()
pvalue

Exercise: overlay chi2 distribution, calculate pvalue from asymptotic

In [ ]:
np.sqrt(nllr_obs)

In [ ]:
def pvalue_to_significance(pvalue):
    return stats.norm.isf(pvalue)

In [ ]:
def significance_to_pvalue(significance):
    return stats.norm.sf(significance)

In [ ]:
significance_to_pvalue(np.sqrt(2 * nllr_obs))

In [ ]:
0.5*(1 - stats.chi2.cdf(2*nllr_obs, 1))

The 0.5 comes intuitively from the fact that around in half of the cases we would have gotten negative signal